<a href="https://colab.research.google.com/github/fernanda-mv/AnalisisdeDatosUQ/blob/main/Taller_GEIH_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Codigo de Maria Fernanda Mora 

In [1]:
#Vamos a importar las librerias necesarias para el analisi de la GEIH
import pandas as pd
import numpy as np

In [2]:
#leyendo los datos de la GEIH 
df_cgsse= pd.read_csv( "https://raw.githubusercontent.com/niconomist98/DataAnalyticsUQ/refs/heads/main/Datos/GEIH2025/cgsse.CSV"
                      ,encoding="latin",sep=";")

df_desocupados= pd.read_csv( "https://raw.githubusercontent.com/niconomist98/DataAnalyticsUQ/refs/heads/main/Datos/GEIH2025/No%20ocupados.CSV"
                      ,encoding="latin",sep=";")    

df_ocupados= pd.read_csv( "https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Ocupados.CSV"
                      ,encoding="latin",sep=";")

df_ft= pd.read_csv( "https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Fuerza%20de%20trabajo.CSV"
                      ,encoding="latin",sep=";")                

/tmp/ipykernel_13710/3859890490.py:8: DtypeWarning: Columns (28,89,133,147,158) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ocupados= pd.read_csv( "https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Ocupados.CSV"
/tmp/ipykernel_13710/3859890490.py:11: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ft= pd.read_csv( "https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Fuerza%20de%20trabajo.CSV"


1. Calculando la poblacion total de colombia:  Para el total nacional calcule :
 
a. población total
b. población en edad de trabajar
c. población que no está en edad de trabajar
d. población económicamente activa
e. población económicamente inactiva
f. tasa de ocupación
g. tasa de desempleo


In [3]:
ptc = df_cgsse["FEX_C18"].sum()
print('Población total de colombia:', int(ptc))

Población total de colombia: 52045001


In [4]:
#Poblacion en edad de trabajar 

sin_edad_trabajar = df_cgsse[df_cgsse['P6040']<15]['FEX_C18'].sum()
edad_trabajar = df_cgsse[df_cgsse['P6040']>=15]['FEX_C18'].sum() #PET


print('La poblacion total sin edad para trabajar es:',int(sin_edad_trabajar))
print('La poblacion total con edad para trabjar es:',int(edad_trabajar))

La poblacion total sin edad para trabajar es: 11391585
La poblacion total con edad para trabjar es: 40653416


In [5]:
#Calculado ahora la poblacion economicamente activa y definimos como poblacion activamente economica
#Tenemos que usar el DSI dado que en el modulo de no ocupados contiene informacion de personas que no estan buscando trabajo 
#No ocupados para este analisis son los que estan buscando trabajo activamente DSI=1 
pea = df_ocupados["FEX_C18"].sum() + df_desocupados[df_desocupados['DSI']==1]["FEX_C18"].sum()

#Tambien podriamos usar la variable FT=1 que es la fuerza de trabajo que cumple con las condiciones 
#Tiene trabajo o esta buscando trabajo

#definimos la poblacion economicamente inactiva como
#Usamos la columna FFT que son las personas fuera de la fuerza de trabajo 
pei = df_ft[df_ft['FFT']==1]['FEX_C18'].sum()

print(f'La poblacion economicamente activa es:', int(pea))
print(f'La poblacion economicamente inactiva es:', int(pei))

La poblacion economicamente activa es: 25974846
La poblacion economicamente inactiva es: 14678569


In [6]:
# Ahora vamos a calcular la tasa de ocupacion (TO)
TO = (df_ocupados['FEX_C18'].sum() / edad_trabajar)

print(f'La tasa de ocupacion: {TO:.2%}')

#Calculando el desempleo (TD) tenemos que

TD = (df_desocupados[df_desocupados['DSI']==1]['FEX_C18'].sum() / pea)
print(f'La tasa de desempleo es {TD:.2%}')

La tasa de ocupacion: 58.43%
La tasa de desempleo es 8.55%


2. Calcule las tasas de desempleo para hombres y mujeres en todos los departamentos de colombia, cuales son los departamentos con mayores y menores tasas de desempleo ? 


In [7]:
# Hacemos merge para tener el sexo en el dataset de desocupados
df_desocupados_gen = pd.merge(
    df_desocupados[df_desocupados["DSI"]==1], 
    df_cgsse[['DIRECTORIO','SECUENCIA_P','ORDEN','P3271']], 
    on=['DIRECTORIO','SECUENCIA_P','ORDEN'], 
    how='left'
)
# Agrupamos por departamento y sexo
desocupados_sexo = (
    df_desocupados_gen.groupby(['DPTO','P3271'])['FEX_C18']
    .sum()
    .reset_index()
    .rename(columns={'FEX_C18':'desocupados'})
)

# Ocupados también con sexo
df_ocupados_gen = pd.merge(
    df_ocupados, 
    df_cgsse[['DIRECTORIO','SECUENCIA_P','ORDEN','P3271']], 
    on=['DIRECTORIO','SECUENCIA_P','ORDEN'], 
    how='left'
)

ocupados_sexo = (
    df_ocupados_gen.groupby(['DPTO','P3271'])['FEX_C18']
    .sum()
    .reset_index()
    .rename(columns={'FEX_C18':'ocupados'})
)

# Unimos ocupados y desocupados para calcular la PEA
pea_sexo = pd.merge(ocupados_sexo, desocupados_sexo, on=['DPTO','P3271'], how='outer').fillna(0)
pea_sexo['PEA'] = pea_sexo['ocupados'] + pea_sexo['desocupados']

# Calculamos la tasa de desempleo (TD)
pea_sexo['TD'] = pea_sexo['desocupados'] / pea_sexo['PEA'] * 100

# Reemplazamos sexo por etiquetas
pea_sexo['Sexo'] = pea_sexo['P3271'].map({1:'Hombres', 2:'Mujeres'})

# Mostramos la tabla final
print(pea_sexo[['DPTO','Sexo','TD']].head(20))

    DPTO     Sexo         TD
0      5  Hombres   4.886691
1      5  Mujeres   9.770415
2      8  Hombres   6.301949
3      8  Mujeres  10.232567
4     11  Hombres   9.821256
5     11  Mujeres   7.078481
6     13  Hombres   4.791098
7     13  Mujeres  12.028807
8     15  Hombres   6.967972
9     15  Mujeres   4.008697
10    17  Hombres   5.583183
11    17  Mujeres   7.677962
12    18  Hombres   2.676220
13    18  Mujeres  15.726909
14    19  Hombres   3.390661
15    19  Mujeres   5.146277
16    20  Hombres   5.991357
17    20  Mujeres  13.209651
18    23  Hombres   5.832048
19    23  Mujeres  19.837285


In [8]:
# Mayor y menor TD por sexo
td_max = pea_sexo.loc[pea_sexo.groupby("Sexo")["TD"].idxmax()]
td_min = pea_sexo.loc[pea_sexo.groupby("Sexo")["TD"].idxmin()]

print("Departamentos con MAYOR tasa de desempleo por sexo:")
print(td_max[['DPTO','Sexo','TD']])

print("\nDepartamentos con MENOR tasa de desempleo por sexo:")
print(td_min[['DPTO','Sexo','TD']])

Departamentos con MAYOR tasa de desempleo por sexo:
    DPTO     Sexo         TD
48    81  Hombres  18.057931
49    81  Mujeres  31.660892

Departamentos con MENOR tasa de desempleo por sexo:
    DPTO     Sexo        TD
56    91  Hombres  1.979606
57    91  Mujeres  1.874288


Podemos ver que el departamento con mayor desempleo lo que va en el 2025 es el 81 que corresponde a Arauca
y con menor desempleo es 91 que es Amazonas 

3. Tomando en consideración únicamente los siguientes grupos de edad: 

18 a 24
24 a 30
30 a 40
40 a 50 
60 en adelante


In [9]:
# 1. Filtrar desocupados en Quindío (63), Risaralda (66) y Caldas (17) ---
df_desocupados_qrc = df_desocupados[
    (df_desocupados["DSI"] == 1) & 
    (df_desocupados["DPTO"].isin([63, 66, 17]))
]

# Calcular total de población desocupada en Q, R, C
PQRC = df_desocupados_qrc["FEX_C18"].sum()
print(f"Población desocupada en Quindío, Risaralda y Caldas: {int(PQRC)}")


Población desocupada en Quindío, Risaralda y Caldas: 103606


In [10]:
# 2. Hacer merge para traer el género (P3271: 1=Hombres, 2=Mujeres) ---
df_desocupados_genero = pd.merge(
    df_desocupados_qrc,
    df_cgsse[["DIRECTORIO", "SECUENCIA_P", "ORDEN", "P3271"]],
    on=["DIRECTORIO", "SECUENCIA_P", "ORDEN"],
    how="left"
)

# 3. Calcular hombres desocupados ---
PHDO = df_desocupados_genero[df_desocupados_genero["P3271"] == 1]["FEX_C18"].sum()
print(f"Hombres desempleados: {int(PHDO)}")

# 4. Calcular mujeres desocupadas ---
PMDO = df_desocupados_genero[df_desocupados_genero["P3271"] == 2]["FEX_C18"].sum()
print(f"Mujeres desempleadas: {int(PMDO)}")

Hombres desempleados: 54096
Mujeres desempleadas: 49509


In [11]:
#Tomando en consideración únicamente los siguientes grupos de edad 
# vamos ara cada uno de los departamentos, cual es el grupo de edad con mayor y menor tasa de desempleo ? 

def clasificar_edad(x):
    if 18 <= x < 24:
        return "18-24"
    elif 24 <= x < 30:
        return "24-30"
    elif 30 <= x < 40:
        return "30-40"
    elif 40 <= x < 50:
        return "40-50"
    elif x >= 60:
        return "60+"
    else:
        return "Otros"  # Para menores de 18 y personas entre 50-59 que no entran

In [13]:
#Despues armamos la PEA por departamento y grupo de edad
#Hacemos merge para traer la edad (P6040) tanto en ocupados como en desocupados.
#Agrupamos por DPTO y grupo_edad.
#Calculamos TD como antes
# Crear columna grupo de edad en df_cgsse
df_cgsse["grupo_edad"] = df_cgsse["P6040"].apply(clasificar_edad)

# --- Desocupados con edad ---
df_desocupados_gen = pd.merge(
    df_desocupados[df_desocupados["DSI"]==1],
    df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","grupo_edad"]],
    on=["DIRECTORIO","SECUENCIA_P","ORDEN"],
    how="left"
)

desocupados_edad = (
    df_desocupados_gen.groupby(["DPTO","grupo_edad"])["FEX_C18"]
    .sum()
    .reset_index()
    .rename(columns={"FEX_C18":"desocupados"})
)

# --- Ocupados con edad ---
df_ocupados_gen = pd.merge(
    df_ocupados,
    df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","grupo_edad"]],
    on=["DIRECTORIO","SECUENCIA_P","ORDEN"],
    how="left"
)

ocupados_edad = (
    df_ocupados_gen.groupby(["DPTO","grupo_edad"])["FEX_C18"]
    .sum()
    .reset_index()
    .rename(columns={"FEX_C18":"ocupados"})
)

# --- PEA y TD ---
pea_edad = pd.merge(ocupados_edad, desocupados_edad, on=["DPTO","grupo_edad"], how="outer").fillna(0)
pea_edad["PEA"] = pea_edad["ocupados"] + pea_edad["desocupados"]
pea_edad["TD"] = pea_edad["desocupados"] / pea_edad["PEA"] * 100


In [19]:
# Filtramos solo los grupos válidos
pea_edad = pea_edad[pea_edad["grupo_edad"].isin(["18-24","24-30","30-40","40-50","60+"])] 

# Para cada departamento, el grupo de mayor y menor TD
mayores = pea_edad.loc[pea_edad.groupby("DPTO")["TD"].idxmax()]
menores = pea_edad.loc[pea_edad.groupby("DPTO")["TD"].idxmin()]

print("Grupo con MAYOR tasa de desempleo por departamento:")
print(mayores[["DPTO","grupo_edad","TD"]])

print("\nGrupo con MENOR tasa de desempleo por departamento:")
print(menores[["DPTO","grupo_edad","TD"]])


Grupo con MAYOR tasa de desempleo por departamento:
     DPTO grupo_edad         TD
0       5      18-24  17.685962
6       8      18-24  22.406918
12     11      18-24  17.797990
18     13      18-24  29.157386
25     15      24-30  14.491221
30     17      18-24   9.972584
36     18      18-24  21.686963
43     19      24-30   7.775308
48     20      18-24  19.042303
54     23      18-24  31.660884
60     25      18-24  15.618810
66     27      18-24  21.102675
72     41      18-24  14.602874
79     44      24-30  24.685637
84     47      18-24  20.675536
90     50      18-24  13.483416
96     52      18-24  16.529172
102    54      18-24  28.074930
109    63      24-30  14.237924
114    66      18-24  15.173556
120    68      18-24  23.222696
127    70      24-30  22.884184
132    73      18-24  19.348020
138    76      18-24  17.821703
144    81      18-24  61.095429
150    85      18-24  27.966955
156    86      18-24  44.878068
163    88      24-30  23.449477
171    91      40-50

En la mayoría de los departamentos, el grupo con mayor tasa de desempleo corresponde a los jóvenes de 18 a 24 años,
lo que refleja las dificultades de inserción laboral de esta población. En algunos casos, como Norte de Santander, 
Valle del Cauca o Bolívar, el grupo más afectado es el de 24 a 30 años, mostrando que la vulnerabilidad también alcanza 
a los adultos jóvenes. Existen excepciones aisladas en otros rangos de edad, pero en general el desempleo juvenil es el 
principal reto en el mercado laboral regional.

4. ¿Cuáles son los 3 departamentos con menor tasa de desempleo de mujeres entre los 18 y los 25 años ? 

In [21]:
# --- Tasas de desempleo de mujeres entre 18 y 24 años para cada departamento ---

# Desempleadas (mujeres 18-24)
desem = (
    df_desocupados_gen[
        (df_desocupados_gen["DSI"] == 1) &
        (df_desocupados_gen["P3271"] == 2) &
        (df_desocupados_gen["P6040"].between(18, 24))
    ]
    .groupby("DPTO")["FEX_C18"].sum()
)

# Población económicamente activa (mujeres 18-24)
persona_activa = (df_ft_gen[
        (df_ft_gen["FT"] == 1) &
        (df_ft_gen["P3271"] == 2) &
        (df_ft_gen["P6040"].between(18, 24))
    ]
    .groupby("DPTO")["FEX_C18"].sum()
)

# Construir DataFrame final (sin 0)
resultado = pd.DataFrame({
    "PEA": persona_activa,
    "Desempleadas": desem
})

resultado["Tasa_desempleo(%)"] = (resultado["Desempleadas"] / resultado["PEA"]) * 100

# Diccionario limpio de nombres de departamentos
dpto_nombre = {
    91: 'AMAZONAS', 81: 'ARAUCA', 5: 'ANTIOQUIA', 8: 'ATLANTICO', 11: 'BOGOTA',
    13: 'BOLIVAR', 15: 'BOYACA', 17: 'CALDAS', 18: 'CAQUETA', 19: 'CAUCA',
    20: 'CESAR', 23: 'CORDOBA', 25: 'CUNDINAMARCA', 27: 'CHOCO', 41: 'HUILA',
    44: 'GUAJIRA', 47: 'MAGDALENA', 50: 'META', 52: 'NARIÑO',
    54: 'NORTE DE SANTANDER', 63: 'QUINDIO', 66: 'RISARALDA', 68: 'SANTANDER',
    70: 'SUCRE', 73: 'TOLIMA', 76: 'VALLE DEL CAUCA', 86: 'PUTUMAYO',
    88: 'SAN ANDRES', 90: 'CASANARE', 94: 'GUAINIA', 95: 'GUAVIARE',
    97: 'VAUPES', 99: 'VICHADA'
}

resultado["Departamento"] = resultado.index.map(dpto_nombre)

# Ordenar de menor a mayor para obtener los 3 con menor tasa
resultado = resultado.sort_values(by="Tasa_desempleo(%)", ascending=True)

print("=== 3 departamentos con menor tasa de desempleo (Mujeres 18-24) ===")
print(resultado.head(3))


=== 3 departamentos con menor tasa de desempleo (Mujeres 18-24) ===
               PEA  Desempleadas  Tasa_desempleo(%) Departamento
DPTO                                                            
63    10289.693612    654.189193           6.357713      QUINDIO
19    37817.442681   2965.201120           7.840829        CAUCA
41    19123.128493   1918.472193          10.032209        HUILA


En el grupo de mujeres entre 18 y 24 años, los departamentos con menor tasa de desempleo son:

1. Quindío con 6,36%, donde 10.289 mujeres conforman la PEA y alrededor de 654 se encuentran desempleadas.

2. Cauca con 7,84%, en una PEA cercana a 37.817 mujeres, de las cuales 2.965 están desempleadas.

3. Huila con 10,03%, con una PEA de 19.123 mujeres y cerca de 1.918 desempleadas.

En conclusion estos tres departamentos muestran mejores condiciones relativas de inserción laboral para mujeres jóvenes en comparación con el resto del país, destacándose el Quindío como el de menor desempleo en este grupo etario.

5. Ahora Vamos a Calcule el promedio de ingresos laborales  de las personas residentes en los 
departamentos de Risaralda, Caldas , Quindío, Antioquia y Valle del Cauca.

In [ ]:
# Filtrar ocupados en los departamentos de interés
deptos_interes = [66, 17, 63, 5, 76]  # 66=Risaralda, 17=Caldas, 63=Quindío, 5=Antioquia, 76=Valle del Cauca

df_ingresos = df_ocupados[df_ocupados["DPTO"].isin(deptos_interes)]

# Calcular el promedio ponderado de ingresos laborales (INGLABO)
promedios_ingresos = (
    df_ingresos.groupby("DPTO")
    .apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum())
    .reset_index(name="Ingreso_promedio")
)

# Reemplazar códigos de departamentos por nombres
mapa_dptos = {66:"Risaralda", 17:"Caldas", 63:"Quindío", 5:"Antioquia", 76:"Valle del Cauca"}
promedios_ingresos["Departamento"] = promedios_ingresos["DPTO"].map(mapa_dptos)

# Reordenar columnas
promedios_ingresos = promedios_ingresos[["Departamento","Ingreso_promedio"]]

print(promedios_ingresos)


      Departamento  Ingreso_promedio
0        Antioquia      2.090819e+06
1           Caldas      1.958683e+06
2          Quindío      1.637336e+06
3        Risaralda      1.621205e+06
4  Valle del Cauca      1.698392e+06


/tmp/ipykernel_1670/522249388.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum())


Los resultados muestran que en 2025 el ingreso laboral promedio más alto entre los departamentos 
analizados se encuentra en Antioquia (≈2,09 millones de pesos), seguido por Caldas (≈1,96 millones).
En contraste, los ingresos más bajos se observan en Risaralda (≈1,62 millones) y Quindío (≈1,63 millones). 
Esto refleja una diferencia regional en el mercado laboral, donde Antioquia lidera en remuneración promedio,
mientras que los departamentos del Eje Cafetero presentan ingresos más modestos.

6. Para los mismos departamentos del punto anterior, calcule el promedio de ingresos laborales de hombres y mujeres en los siguientes rangos de edad : 
-18 a 24
-24 a 30
-30 a 40
-40 a 50 
-60 en adelante 

In [20]:
# Definir función para agrupar edades
def clasificar_edad(x):
    if 18 <= x < 24:
        return "18-24"
    elif 24 <= x < 30:
        return "24-30"
    elif 30 <= x < 40:
        return "30-40"
    elif 40 <= x < 50:
        return "40-50"
    elif x >= 60:
        return "60+"
    else:
        return None

# Crear variable de grupo de edad
df_cgsse["grupo_edad"] = df_cgsse["P6040"].apply(clasificar_edad)

In [21]:
# Merge con ocupados para traer sexo y edad
df_ocupados_gen = pd.merge(
    df_ocupados,
    df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","P3271","grupo_edad"]],
    on=["DIRECTORIO","SECUENCIA_P","ORDEN"],
    how="left"
)

# Filtrar departamentos de interés
deptos_interes = [66, 17, 63, 5, 76]
df_filtrado = df_ocupados_gen[df_ocupados_gen["DPTO"].isin(deptos_interes)]

In [22]:
# Calcular promedio ponderado de ingresos por depto, sexo y grupo de edad
prom_ingresos = (
    df_filtrado.groupby(["DPTO","P3271","grupo_edad"])
    .apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum())
    .reset_index(name="Ingreso_promedio")
)

# Mapear nombres de departamentos y sexo
mapa_dptos = {66:"Risaralda", 17:"Caldas", 63:"Quindío", 5:"Antioquia", 76:"Valle del Cauca"}
prom_ingresos["Departamento"] = prom_ingresos["DPTO"].map(mapa_dptos)
prom_ingresos["Sexo"] = prom_ingresos["P3271"].map({1:"Hombres", 2:"Mujeres"})

# Reordenar columnas
prom_ingresos = prom_ingresos[["Departamento","Sexo","grupo_edad","Ingreso_promedio"]]

print(prom_ingresos.head(20))

   Departamento     Sexo grupo_edad  Ingreso_promedio
0     Antioquia  Hombres      18-24      1.364005e+06
1     Antioquia  Hombres      24-30      1.828144e+06
2     Antioquia  Hombres      30-40      2.159206e+06
3     Antioquia  Hombres      40-50      2.771747e+06
4     Antioquia  Hombres        60+      1.738620e+06
5     Antioquia  Mujeres      18-24      1.418639e+06
6     Antioquia  Mujeres      24-30      1.837916e+06
7     Antioquia  Mujeres      30-40      2.291801e+06
8     Antioquia  Mujeres      40-50      2.111094e+06
9     Antioquia  Mujeres        60+      1.510449e+06
10       Caldas  Hombres      18-24      2.216063e+06
11       Caldas  Hombres      24-30      1.754677e+06
12       Caldas  Hombres      30-40      1.762301e+06
13       Caldas  Hombres      40-50      2.470039e+06
14       Caldas  Hombres        60+      2.309864e+06
15       Caldas  Mujeres      18-24      1.133456e+06
16       Caldas  Mujeres      24-30      1.832716e+06
17       Caldas  Mujeres    

/tmp/ipykernel_1670/4127443079.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum())


Los resultados muestran que los ingresos laborales aumentan con la edad en la mayoría de los casos, alcanzando su punto más alto en el grupo de 40 a 50 años, tanto para hombres como para mujeres. También se observa que, salvo excepciones puntuales como en Caldas (jóvenes hombres 18–24), los hombres suelen tener ingresos promedio superiores a las mujeres en los mismos rangos de edad, lo que refleja una brecha salarial de género en los departamentos analizados.

7. Basado en la siguiente lista de códigos de oficios : 
¿Cuáles son los oficios con mayores promedios salariales a nivel nacional en general  ? 


In [24]:
print(df_ocupados.columns)


Index(['PERIODO', 'MES', 'PER', 'DIRECTORIO', 'SECUENCIA_P', 'ORDEN', 'HOGAR',
       'REGIS', 'AREA', 'CLASE',
       ...
       'P1881', 'P1881S1', 'P1882', 'P7240', 'P7240S1', 'OCI', 'INGLABO',
       'RAMA2D_R4', 'RAMA4D_R4', 'OFICIO_C8'],
      dtype='object', length=202)


In [29]:
# Calcular promedio ponderado de ingresos laborales por oficio (CIUO-08)
ingresos_ciuo = (
    df_ocupados.groupby("OFICIO_C8")
    .apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum())
    .reset_index(name="Ingreso_promedio")
)

# Ordenar de mayor a menor ingreso
ingresos_ciuo = ingresos_ciuo.sort_values("Ingreso_promedio", ascending=False)

# Mostrar los 10 oficios mejor pagados
print(ingresos_ciuo.head(10))



     OFICIO_C8  Ingreso_promedio
63        2267      5.176025e+07
25        1349      1.673692e+07
93        2523      1.100000e+07
96        2612      1.040657e+07
94        2529      1.015048e+07
156       3321      9.351115e+06
52        2212      8.946374e+06
7         1211      8.923134e+06
90        2513      8.852754e+06
12        1222      8.541606e+06


/tmp/ipykernel_1670/1259098874.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum())


Los oficios con mayores ingresos promedio en Colombia en 2025, según la GEIH, son:

2267 – Conductores de camiones pesados y de alto tonelaje: sorprende al encabezar con un ingreso promedio superior a 51 millones de pesos, aunque es probable que corresponda a un error de registro o a valores atípicos, pues no es coherente con la estructura salarial real.

1349 – Gerentes de servicios profesionales no clasificados: con ingresos de alrededor de 16,7 millones, representan altos cargos directivos en servicios especializados.

2523 – Profesionales en análisis de bases de datos y redes: con ingresos cercanos a 11 millones, reflejan la alta demanda de perfiles en tecnología y análisis de información.

2612 – Jueces y 2529 – Desarrolladores de software: ambos rondan los 10 millones, confirmando la relevancia de la justicia y las TIC como sectores de alta remuneración.

3321 – Agentes de bolsa y financieros: con ingresos de 9,3 millones, destacan en el sector financiero.

2212 – Médicos especialistas: con salarios de casi 9 millones, confirman la alta remuneración del sector salud.

1211 – Directores generales y 1222 – Gerentes de finanzas: ambos sobre los 8,5 millones, ratifican que los cargos de dirección se mantienen entre los mejor pagados.

En conclusion los mayores ingresos se concentran en cargos directivos, justicia, salud y tecnología, con casos atípicos en oficios operativos como transporte pesado, que probablemente responden a registros específicos o valores extremos en la encuesta

8. Elija un código de oficio y , basado en los datos, concluya si  existen diferencias significativas entre los promedios  salariales de hombres y mujeres para dicho oficio en los departamentos de Caldas , Quindío, Antioquia y Valle del Cauca.


In [ ]:
# Oficio elegido (2212 = Médicos especialistas)
codigo_oficio = 2212

# Departamentos de interés
deptos_interes = [17, 63, 5, 76]

# Merge para traer sexo (P3271) desde df_cgsse
df_ocupados_gen = pd.merge(
    df_ocupados,
    df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","P3271"]],
    on=["DIRECTORIO","SECUENCIA_P","ORDEN"],
    how="left"
)

# Filtrar por oficio y departamentos
df_filtrado = df_ocupados_gen[
    (df_ocupados_gen["OFICIO_C8"] == codigo_oficio) &
    (df_ocupados_gen["DPTO"].isin(deptos_interes))
]

# Calcular promedio ponderado de ingresos por sexo y departamento
promedios = (
    df_filtrado.groupby(["DPTO","P3271"])
    .apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum())
    .reset_index(name="Ingreso_promedio")
)

# Mapear nombres
mapa_dptos = {17:"Caldas", 63:"Quindío", 5:"Antioquia", 76:"Valle del Cauca"}
promedios["Departamento"] = promedios["DPTO"].map(mapa_dptos)
promedios["Sexo"] = promedios["P3271"].map({1:"Hombres", 2:"Mujeres"})

# Reordenar columnas
promedios = promedios[["Departamento","Sexo","Ingreso_promedio"]]

print(promedios)


      Departamento     Sexo  Ingreso_promedio
0        Antioquia  Hombres      1.568818e+07
1           Caldas  Hombres      4.913083e+06
2           Caldas  Mujeres      1.000000e+07
3          Quindío  Hombres      4.693600e+06
4          Quindío  Mujeres      6.563233e+06
5  Valle del Cauca  Hombres      7.000000e+06
6  Valle del Cauca  Mujeres      1.291129e+07


/tmp/ipykernel_1670/3257977744.py:24: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum())


Los resultados muestran diferencias claras entre hombres y mujeres en el oficio analizado. En Antioquia los hombres presentan ingresos mucho más altos (≈15,7 millones). En contraste, en Caldas, Quindío y Valle del Cauca las mujeres superan a los hombres, llegando incluso a duplicar sus ingresos en algunos casos (ej. Valle del Cauca: 12,9 millones vs. 7 millones).

En conclusión, sí existen diferencias significativas entre géneros, pero no son uniformes: en unos departamentos predominan los hombres y en otros las mujeres, lo que evidencia una brecha salarial variable por territorio dentro del mismo oficio.

9. En el departamento del Quindío cual es el oficio con mayor y menor  promedio salarial en hombres y mujeres? 

In [31]:
# Filtrar ocupados del Quindío y unir sexo
df_ocupados_gen = pd.merge(
    df_ocupados,
    df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","P3271"]],
    on=["DIRECTORIO","SECUENCIA_P","ORDEN"],
    how="left"
)

df_quindio = df_ocupados_gen[df_ocupados_gen["DPTO"] == 63]

# Calcular promedio ponderado por oficio y sexo
prom_quindio = (
    df_quindio.groupby(["OFICIO_C8","P3271"])
    .apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum())
    .reset_index(name="Ingreso_promedio")
)

# Mapear sexo
prom_quindio["Sexo"] = prom_quindio["P3271"].map({1:"Hombres", 2:"Mujeres"})

# Oficio con mayor y menor ingreso para hombres
max_hombres = prom_quindio[prom_quindio["Sexo"]=="Hombres"].sort_values("Ingreso_promedio", ascending=False).head(1)
min_hombres = prom_quindio[prom_quindio["Sexo"]=="Hombres"].sort_values("Ingreso_promedio", ascending=True).head(1)

# Oficio con mayor y menor ingreso para mujeres
max_mujeres = prom_quindio[prom_quindio["Sexo"]=="Mujeres"].sort_values("Ingreso_promedio", ascending=False).head(1)
min_mujeres = prom_quindio[prom_quindio["Sexo"]=="Mujeres"].sort_values("Ingreso_promedio", ascending=True).head(1)

print("Mayor salario Hombres:\n", max_hombres)
print("\nMenor salario Hombres:\n", min_hombres)
print("\nMayor salario Mujeres:\n", max_mujeres)
print("\nMenor salario Mujeres:\n", min_mujeres)


Mayor salario Hombres:
     OFICIO_C8  P3271  Ingreso_promedio     Sexo
24       2142      1      7.986427e+06  Hombres

Menor salario Hombres:
      OFICIO_C8  P3271  Ingreso_promedio     Sexo
183       7111      1               0.0  Hombres

Mayor salario Mujeres:
     OFICIO_C8  P3271  Ingreso_promedio     Sexo
35       2212      2      6.563233e+06  Mujeres

Menor salario Mujeres:
      OFICIO_C8  P3271  Ingreso_promedio     Sexo
206       7513      2               0.0  Mujeres


/tmp/ipykernel_1670/2031362416.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: (x["INGLABO"] * x["FEX_C18"]).sum() / x["FEX_C18"].sum())


En el Quindío, el oficio mejor remunerado para los hombres es 2142 – Ingenieros electrónicos con un ingreso promedio de 7,98 millones de pesos.

El oficio con menor remuneración entre los hombres es 7111 – Albañiles, que aparece con ingreso cero, lo cual probablemente refleja ausencia de registros válidos o subdeclaración de ingresos en la muestra.

Para las mujeres, el mayor ingreso corresponde a 2212 – Médicos especialistas, con un promedio de 6,56 millones de pesos.

El menor ingreso entre las mujeres se observa en 7513 – Trabajadoras de preparación de alimentos callejeros, también con valor cero, lo que igualmente refleja falta de ingresos reportados en la encuesta.

Podemos decir que en el Quindío, los oficios mejor remunerados difieren por género (ingeniería electrónica en hombres, medicina especializada en mujeres), mientras que los más bajos muestran problemas de informalidad o subregistro en sectores como construcción y comercio informal de alimentos.